In [1]:
import pandas as pd

Trabajo en el siguiente rango de dias:

In [2]:
firstDay = pd.Timestamp(2019, 4, 24)
lastDay = pd.Timestamp(2019, 4, 27)

Los datos son los siguientes

In [3]:
location = 'data/'
clicksFile = location + 'clicks.csv'

Cargo los datos de las auctions en el rango de dias indicado

In [4]:
dtype = {
    'advertiser_id': 'category',
    'source_id': 'category',
    'created': 'str',
    'latitude': 'float32',
    'longitude': 'float32',
    'wifi_connection': 'bool',
    'carrier_id': 'category',
    'os_minor': 'category',
    'os_major': 'category',
    'agent_device': 'category',
    'specs_brand': 'category',
    'brand': 'category',
    'timeToClick': 'float32',
    'touchX': 'str',
    'touchY': 'str',
    'ref_type': 'category',
    'ref_hash': 'int64',
}

In [5]:
clicks = pd.read_csv(clicksFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['created'])

In [6]:
clicksIsInDateInterval = (clicks['created'] >= firstDay) & (clicks['created'] < lastDay)
clicks = clicks[clicksIsInDateInterval]

In [7]:
clicks = clicks.drop(clicks[clicks['touchX'] == 'Infinity'].index)

In [8]:
clicks['touchX'] = clicks['touchX'].astype('float32')

In [9]:
clicks['touchY'] = clicks['touchY'].astype('float32')

In [10]:
clicks['created'] = clicks['created'].dt.tz_localize(None)

In [11]:
clicks = clicks.rename(columns={'ref_hash': 'device_id'})

In [12]:
clicks['seconds'] = (clicks['created'] - firstDay).dt.total_seconds()
clicks['day'] = clicks['created'].dt.day - firstDay.day + 1

In [13]:
clicks.head()

,advertiser_id,source_id,created,latitude,longitude,wifi_connection,carrier_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_type,device_id,seconds,day
20,1,0,2019-04-25 13:47:20.817,1.714547,0.871535,False,9.0,6.795761880764845e+18,NaN,5.131615556736863e+18,71913840936116953,NaN,4.706000,0.910,0.036,1891515180541284343,7727410219160218534,136040.817,2
21,1,1,2019-04-25 16:16:56.642,1.686080,0.880132,True,1.0,7.531669329342818e+18,NaN,5.648867414868049e+18,3576558787748411622,NaN,3.620000,0.577,4.626,1891515180541284343,1054780916525454915,145016.642,2
22,1,1,2019-04-25 16:58:53.030,1.714512,0.871062,True,1.0,3.575963029724781e+18,NaN,5.754947116114108e+18,3576558787748411622,NaN,39.522999,0.816,0.875,1891515180541284343,4412003964704268712,147533.030,2
23,1,3,2019-04-25 16:30:33.253,1.747826,0.864290,True,7.0,3.575963029724781e+18,NaN,5.754947116114108e+18,71913840936116953,NaN,0.919000,0.956,0.069,1891515180541284343,3891350874593022574,145833.253,2
24,1,1,2019-04-25 17:02:30.824,1.730828,0.855571,True,1.0,5.310344816890522e+18,NaN,3.581232574980917e+18,3576558787748411622,NaN,128.764999,0.613,0.709,1891515180541284343,4541657187949816744,147750.824,2


In [14]:
clicks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16927 entries, 20 to 61878
Data columns (total 19 columns):
advertiser_id      16927 non-null category
source_id          16927 non-null category
created            16927 non-null datetime64[ns]
latitude           16927 non-null float32
longitude          16927 non-null float32
wifi_connection    16927 non-null bool
carrier_id         16650 non-null category
os_minor           16920 non-null category
agent_device       2030 non-null category
os_major           16920 non-null category
specs_brand        16927 non-null category
brand              4009 non-null category
timeToClick        9497 non-null float32
touchX             11343 non-null float32
touchY             11343 non-null float32
ref_type           16927 non-null category
device_id          16927 non-null int64
seconds            16927 non-null float64
day                16927 non-null int64
dtypes: bool(1), category(9), datetime64[ns](1), float32(5), float64(1), int64(2)
memo

## Busqueda de features

In [15]:
features = pd.DataFrame(clicks['device_id'].unique(), columns=['device_id'])

In [16]:
features.head()

,device_id
0,7727410219160218534
1,1054780916525454915
2,4412003964704268712
3,3891350874593022574
4,4541657187949816744


In [17]:
clicksCount = clicks[['device_id']].groupby(by=['device_id']).size()
clicksCount = clicksCount.reset_index().rename(columns={0: 'clicksCount'})
features = features.merge(clicksCount, how='left')

In [18]:
clicksMostFreqDay = clicks[['device_id', 'day']].groupby(by=['device_id']).agg(lambda x: x.value_counts().index[0])
clicksMostFreqDay = clicksMostFreqDay.reset_index().rename(columns={'day': 'clicksMostFreqDay'})
features = features.merge(clicksMostFreqDay, how='left')

In [19]:
clicksMeanInterval = clicks[['device_id', 'seconds']].sort_values(by='seconds').groupby(by='device_id').agg(lambda x: (x['seconds'] - x['seconds'].shift(1).fillna(0)).mean())
clicksMeanInterval = clicksMeanInterval.reset_index().rename(columns={'seconds': 'clicksMeanInterval'})
features = features.merge(clicksMeanInterval, how='left')

In [20]:
features['clicksAproxFreq'] = features['clicksMeanInterval']**-1

In [21]:
clicksStdDevInterval = clicks[['device_id', 'seconds']].sort_values(by='seconds').groupby(by='device_id').agg(lambda x: (x['seconds'] - x['seconds'].shift(1).fillna(0)).std())
clicksStdDevInterval = clicksStdDevInterval.reset_index().rename(columns={'seconds': 'clicksStdDevInterval'})
features = features.merge(clicksStdDevInterval, how='left')

In [22]:
clicksLast = clicks[['device_id', 'seconds']].groupby(by='device_id').max()
clicksLast = clicksLast.reset_index().rename(columns={'seconds': 'clicksLast'})
features = features.merge(clicksLast, how='left')

In [23]:
features.head()

,device_id,clicksCount,clicksMostFreqDay,clicksMeanInterval,clicksAproxFreq,clicksStdDevInterval,clicksLast
0,7727410219160218534,3,2,45997.182667,0.000022,77981.990853,137991.548
1,1054780916525454915,11,2,22061.224364,0.000045,14027.039449,242673.468
2,4412003964704268712,1,2,147533.030000,0.000007,NaN,147533.030
3,3891350874593022574,3,2,48611.084333,0.000021,40846.527403,145833.253
4,4541657187949816744,2,2,73875.412000,0.000014,104270.830038,147750.824


In [24]:
features.to_csv('data/clicksFeatures7.csv', index=False)